In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import joblib
import sklearn
from sklearn.svm import SVR
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [6]:
df = pd.read_csv('../../dataset/hao_dataset.csv')
df

doi                                  in_source  \
0       10.1016/j.jcis.2010.06.037                       chloroindium oxalate   
1              10.1021/j100019a063                       chloroindium oxalate   
2              10.1021/j100019a063                       chloroindium oxalate   
3              10.1021/j100019a063                       chloroindium oxalate   
4              10.1021/j100070a004                       chloroindium oxalate   
..                             ...                                        ...   
215          10.1557/jmr.2006.0068                    indium trifluoroacetate   
216          10.1557/jmr.2006.0068                    indium trifluoroacetate   
217          10.1557/jmr.2006.0068                    indium trifluoroacetate   
218  10.1021/acs.chemmater.7b04069  indium tris(N,N-diisopropylacetamidinato)   
219  10.1021/acs.chemmater.7b04069  indium tris(N,N-diisopropylacetamidinato)   

     in_amount_mmol                       p_source  p_amount_mmol         sol  \
0              4.20  tris(trimethylsilyl)phosphine           2.07        None   
1              0.88  tris(trimethylsilyl)phosphine           0.52        None   
2              1.76  tris(trimethylsilyl)phosphine           1.04        None   
3              1.76  tris(trimethylsilyl)phosphine           1.04        None   
4              0.88  tris(trimethylsilyl)phosphine           1.40        None   
..              ...                            ...            ...         ...   
215            0.11  tris(trimethylsilyl)phosphine           0.10  octadecene   
216            0.11  tris(trimethylsilyl)phosphine           0.10  octadecene   
217            0.11  tris(trimethylsilyl)phosphine           0.10  octadecene   
218            0.30  tris(trimethylsilyl)phosphine           0.15  mesitylene   
219            0.30  tris(trimethylsilyl)phosphine           0.15  mesitylene   

     sol_amount_ml                TOP  TOP_amount_mmol           acid  ...  \
0              0.0  trioctylphosphine        13.452570           None  ...   
1              0.0  trioctylphosphine        13.490343           None  ...   
2              0.0  trioctylphosphine         5.396137           None  ...   
3              0.0  trioctylphosphine         5.396137           None  ...   
4              0.0  trioctylphosphine        13.490343           None  ...   
..             ...                ...              ...            ...  ...   
215            7.0               None         0.000000  myristic acid  ...   
216            7.0               None         0.000000  myristic acid  ...   
217            7.0               None         0.000000  myristic acid  ...   
218            5.0               None         0.000000  palmitic acid  ...   
219            5.0               None         0.000000  palmitic acid  ...   

                       other other_amount_mmol  total_volume_ml temp_c  \
0    trioctylphosphine oxide         11.111111           17.112    280   
1    trioctylphosphine oxide          5.555556           11.573    270   
2    trioctylphosphine oxide          0.222222            2.629    270   
3    trioctylphosphine oxide          0.222222            2.629    300   
4    trioctylphosphine oxide          5.555556           11.573    270   
..                       ...               ...              ...    ...   
215                     None          0.000000            7.000    270   
216                     None          0.000000            7.000    270   
217                     None          0.000000            7.000    270   
218                     None          0.000000            5.077    150   
219                     None          0.000000            5.077    230   

     time_min diameter_nm  abs_nm emission_nm  date_input  user  
0      4320.0        None     599         612   6/25/2021   Hao  
1      4320.0         2.6     530        None   6/25/2021   Hao  
2      4320.0         3.5     595        None   6/25/2021   H

In [7]:
#Checks if there are any columns with no values
df.isna().sum()

doi                  0
in_source            0
in_amount_mmol       0
p_source             0
p_amount_mmol        0
sol                  0
sol_amount_ml        0
TOP                  0
TOP_amount_mmol      0
acid                 0
acid_amount_mmol     0
amine                0
amine_amount_mmol    0
thiol                0
thiol_amount_mmol    0
zinc                 0
zinc_amount_mmol     0
other                0
other_amount_mmol    0
total_volume_ml      0
temp_c               0
time_min             0
diameter_nm          0
abs_nm               0
emission_nm          0
date_input           0
user                 0
dtype: int64

In [8]:
#Separate out initial DataFrame into the input features and output features
df_input = df.drop(columns =['diameter_nm', 'abs_nm', 'emission_nm','doi','user','date_input'], inplace = False, axis = 1)
df_output = df[['diameter_nm', 'abs_nm', 'emission_nm']]

In [9]:
#Checks the column names, and ensures that they do not have any leading or trailing spaces
df_input.columns = df_input.columns.str.strip()
df_output.columns = df_output.columns.str.strip()

In [10]:
df_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   in_source          220 non-null    object 
 1   in_amount_mmol     220 non-null    float64
 2   p_source           220 non-null    object 
 3   p_amount_mmol      220 non-null    float64
 4   sol                220 non-null    object 
 5   sol_amount_ml      220 non-null    float64
 6   TOP                220 non-null    object 
 7   TOP_amount_mmol    220 non-null    float64
 8   acid               220 non-null    object 
 9   acid_amount_mmol   220 non-null    float64
 10  amine              220 non-null    object 
 11  amine_amount_mmol  220 non-null    float64
 12  thiol              220 non-null    object 
 13  thiol_amount_mmol  220 non-null    float64
 14  zinc               220 non-null    object 
 15  zinc_amount_mmol   220 non-null    float64
 16  other              220 non

In [11]:
df_input['temp_c'] = df_input['temp_c'].astype(float)
df_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   in_source          220 non-null    object 
 1   in_amount_mmol     220 non-null    float64
 2   p_source           220 non-null    object 
 3   p_amount_mmol      220 non-null    float64
 4   sol                220 non-null    object 
 5   sol_amount_ml      220 non-null    float64
 6   TOP                220 non-null    object 
 7   TOP_amount_mmol    220 non-null    float64
 8   acid               220 non-null    object 
 9   acid_amount_mmol   220 non-null    float64
 10  amine              220 non-null    object 
 11  amine_amount_mmol  220 non-null    float64
 12  thiol              220 non-null    object 
 13  thiol_amount_mmol  220 non-null    float64
 14  zinc               220 non-null    object 
 15  zinc_amount_mmol   220 non-null    float64
 16  other              220 non

In [12]:
#Initializes 2 lists to contain all of the numerical and categorical input columns
input_num_cols = [col for col in df_input.columns if df[col].dtypes !='O']
input_cat_cols = [col for col in df_input.columns if df[col].dtypes =='O']

In [13]:
input_cat_cols

['in_source',
 'p_source',
 'sol',
 'TOP',
 'acid',
 'amine',
 'thiol',
 'zinc',
 'other']

In [14]:
#Initializes the ColumnTransformer object, and specifies what it will do with a passed in dataframe
ct = ColumnTransformer([
    ('step1', StandardScaler(), input_num_cols),
    ('step2', OneHotEncoder(sparse=False, handle_unknown='ignore'), input_cat_cols)
], remainder = 'passthrough')

In [15]:
#Uses the ColumnTransformer object to modify the input columns
df_input_scaled_encoded = pd.DataFrame(ct.fit_transform(df_input))
df_input_scaled_encoded

0         1         2         3         4         5         6   \
0    2.794105  0.773217 -0.528734  1.957308 -0.437274 -0.642624 -0.196651   
1    0.050983 -0.359655 -0.528734  1.963657 -0.437274 -0.642624 -0.196651   
2    0.778075  0.020405 -0.528734  0.603121 -0.437274 -0.642624 -0.196651   
3    0.778075  0.020405 -0.528734  0.603121 -0.437274 -0.642624 -0.196651   
4    0.050983  0.283524 -0.528734  1.963657 -0.437274 -0.642624 -0.196651   
..        ...       ...       ...       ...       ...       ...       ...   
215 -0.585223 -0.666627 -0.070675 -0.303903 -0.323598 -0.642624 -0.196651   
216 -0.585223 -0.666627 -0.070675 -0.303903 -0.323598 -0.642624 -0.196651   
217 -0.585223 -0.666627 -0.070675 -0.303903 -0.323598 -0.642624 -0.196651   
218 -0.428237 -0.630083 -0.201549 -0.303903 -0.096248 -0.600711 -0.196651   
219 -0.428237 -0.630083 -0.201549 -0.303903 -0.096248 -0.600711 -0.196651   

          7         8         9   ...   62   63   64   65   66   67   68   69  \
0   -0.62702  8.027908  0.414520  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0   
1   -0.62702  3.882472  0.064420  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0   
2   -0.62702 -0.097146 -0.500900  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0   
3   -0.62702 -0.097146 -0.500900  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0   
4   -0.62702  3.882472  0.064420  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0   
..       ...       ...       ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
215 -0.62702 -0.262964 -0.224624  ...  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
216 -0.62702 -0.262964 -0.224624  ...  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
217 -0.62702 -0.262964 -0.224624  ...  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
218 -0.62702 -0.262964 -0.346170  ...  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
219 -0.62702 -0.262964 -0.346170  ...  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

      70   71  
0    0.0  0.0  
1    0.0  0.0  
2    0.0  0.0  
3    0.0  0.0  
4    0.0  0.0  
..   ...  ...  
215  0.0  0.0  
216  0.0  0.0  
217  0.0  0.0  
218  0.0  0.0  
219  0.0  0.0  

[220 rows x 72 columns]

In [16]:
ct.transformers_

[('step1',
  StandardScaler(),
  ['in_amount_mmol',
   'p_amount_mmol',
   'sol_amount_ml',
   'TOP_amount_mmol',
   'acid_amount_mmol',
   'amine_amount_mmol',
   'thiol_amount_mmol',
   'zinc_amount_mmol',
   'other_amount_mmol',
   'total_volume_ml',
   'temp_c',
   'time_min']),
 ('step2',
  OneHotEncoder(handle_unknown='ignore', sparse=False),
  ['in_source',
   'p_source',
   'sol',
   'TOP',
   'acid',
   'amine',
   'thiol',
   'zinc',
   'other'])]

In [17]:
#Look into ct object to see the column titles
array_of_cat_titles = ct.transformers_[1][1].get_feature_names()
len(array_of_cat_titles) #There are a total of 41 encoded categorical columns

60

In [18]:
#Number of numerical columns in the dataset
len(input_num_cols)

12

In [19]:
for i in range(len(input_num_cols)):
    df_input_scaled_encoded.rename(columns={df_input_scaled_encoded.columns[i]: input_num_cols[i]}, inplace = True)

for j in range(len(array_of_cat_titles)):
    df_input_scaled_encoded.rename(columns={df_input_scaled_encoded.columns[i+1]: array_of_cat_titles[j]}, inplace = True)
    i = i + 1

In [20]:
df_input_scaled_encoded

in_amount_mmol  p_amount_mmol  sol_amount_ml  TOP_amount_mmol  \
0          2.794105       0.773217      -0.528734         1.957308   
1          0.050983      -0.359655      -0.528734         1.963657   
2          0.778075       0.020405      -0.528734         0.603121   
3          0.778075       0.020405      -0.528734         0.603121   
4          0.050983       0.283524      -0.528734         1.963657   
..              ...            ...            ...              ...   
215       -0.585223      -0.666627      -0.070675        -0.303903   
216       -0.585223      -0.666627      -0.070675        -0.303903   
217       -0.585223      -0.666627      -0.070675        -0.303903   
218       -0.428237      -0.630083      -0.201549        -0.303903   
219       -0.428237      -0.630083      -0.201549        -0.303903   

     acid_amount_mmol  amine_amount_mmol  thiol_amount_mmol  zinc_amount_mmol  \
0           -0.437274          -0.642624          -0.196651          -0.62702   
1           -0.437274          -0.642624          -0.196651          -0.62702   
2           -0.437274          -0.642624          -0.196651          -0.62702   
3           -0.437274          -0.642624          -0.196651          -0.62702   
4           -0.437274          -0.642624          -0.196651          -0.62702   
..                ...                ...                ...               ...   
215         -0.323598          -0.642624          -0.196651          -0.62702   
216         -0.323598          -0.642624          -0.196651          -0.62702   
217         -0.323598          -0.642624          -0.196651          -0.62702   
218         -0.096248          -0.600711          -0.196651          -0.62702   
219         -0.096248          -0.600711          -0.196651          -0.62702   

     other_amount_mmol  total_volume_ml  ...  x8_None  x8_acetic acid  \
0             8.027908         0.414520  ...      0.0             0.0   
1             3.882472         0.064420  ...      0.0             0.0   
2            -0.097146        -0.500900  ...      0.0             0.0   
3            -0.097146        -0.500900  ...      0.0             0.0   
4             3.882472         0.064420  ...      0.0             0.0   
..                 ...              ...  ...      ...             ...   
215          -0.262964        -0.224624  ...      1.0             0.0   
216          -0.262964        -0.224624  ...      1.0             0.0   
217          -0.262964        -0.224624  ...      1.0             0.0   
218          -0.262964        -0.346170  ...      1.0             0.0   
219          -0.262964        -0.346170  ...      1.0             0.0   

     x8_copper bromide  x8_oleic acid  x8_superhydride  \
0                  0.0            0.0              0.0   
1                  0.0            0.0              0.0   
2                  0.0            0.0              0.0   
3                  0.0            0.0              0.0   
4                  0.0            0.0              0.0   
..                 ...            ...              ...   
215                0.0            0.0              0.0   
216                0.0            0.0              0.0   
217                0.0            0.0              0.0   
218                0.0            0.0              0.0   
219                0.0            0.0              0.0   

     x8_tetrabutylammonium myristate  x8_trioctylamine  \
0                                0.0               0.0   
1                                0.0               0.0   
2                                0.0               0.0   
3                                0.0               0.0   
4                                0.0               0.0   
..                               ...               ...   
215                              0.0               0.0   
216                              0.0               0.0   
217                              0.0               0.0   
218                              0.0 

In [21]:
df_scaled_encoded = pd.concat([df_input_scaled_encoded, df_output], axis = 1)
df_scaled_encoded

in_amount_mmol  p_amount_mmol  sol_amount_ml  TOP_amount_mmol  \
0          2.794105       0.773217      -0.528734         1.957308   
1          0.050983      -0.359655      -0.528734         1.963657   
2          0.778075       0.020405      -0.528734         0.603121   
3          0.778075       0.020405      -0.528734         0.603121   
4          0.050983       0.283524      -0.528734         1.963657   
..              ...            ...            ...              ...   
215       -0.585223      -0.666627      -0.070675        -0.303903   
216       -0.585223      -0.666627      -0.070675        -0.303903   
217       -0.585223      -0.666627      -0.070675        -0.303903   
218       -0.428237      -0.630083      -0.201549        -0.303903   
219       -0.428237      -0.630083      -0.201549        -0.303903   

     acid_amount_mmol  amine_amount_mmol  thiol_amount_mmol  zinc_amount_mmol  \
0           -0.437274          -0.642624          -0.196651          -0.62702   
1           -0.437274          -0.642624          -0.196651          -0.62702   
2           -0.437274          -0.642624          -0.196651          -0.62702   
3           -0.437274          -0.642624          -0.196651          -0.62702   
4           -0.437274          -0.642624          -0.196651          -0.62702   
..                ...                ...                ...               ...   
215         -0.323598          -0.642624          -0.196651          -0.62702   
216         -0.323598          -0.642624          -0.196651          -0.62702   
217         -0.323598          -0.642624          -0.196651          -0.62702   
218         -0.096248          -0.600711          -0.196651          -0.62702   
219         -0.096248          -0.600711          -0.196651          -0.62702   

     other_amount_mmol  total_volume_ml  ...  x8_oleic acid  x8_superhydride  \
0             8.027908         0.414520  ...            0.0              0.0   
1             3.882472         0.064420  ...            0.0              0.0   
2            -0.097146        -0.500900  ...            0.0              0.0   
3            -0.097146        -0.500900  ...            0.0              0.0   
4             3.882472         0.064420  ...            0.0              0.0   
..                 ...              ...  ...            ...              ...   
215          -0.262964        -0.224624  ...            0.0              0.0   
216          -0.262964        -0.224624  ...            0.0              0.0   
217          -0.262964        -0.224624  ...            0.0              0.0   
218          -0.262964        -0.346170  ...            0.0              0.0   
219          -0.262964        -0.346170  ...            0.0              0.0   

     x8_tetrabutylammonium myristate  x8_trioctylamine  \
0                                0.0               0.0   
1                                0.0               0.0   
2                                0.0               0.0   
3                                0.0               0.0   
4                                0.0               0.0   
..                               ...               ...   
215                              0.0               0.0   
216                              0.0               0.0   
217                              0.0               0.0   
218                              0.0               0.0   
219                              0.0               0.0   

     x8_trioctylphosphine oxide  x8_water  x8_zinc iodide  diameter_nm  \
0                           1.0       0.0             0.0         None   
1                           1.0       0.0             0.0          2.6   
2                           1.0       0.0             0.0          3.5   
3                           1.0       0.0             0.0          4.6   
4                           1.0       0.0             0.0         2.52   
..                          ...       ...             ...          ...   
215            

In [22]:
df_scaled_encoded.to_csv('hao_dataset_scaled.csv')

In [23]:
#Saves the row indexes to drop for absorbance modelling into a list
total_row_num = len(df_scaled_encoded)
drop_list_abs =[]
for row_i in range(total_row_num):
    if df_scaled_encoded['abs_nm'].values[row_i] == 'None':
        drop_list_abs.append(row_i)
    
len(drop_list_abs)

14

In [24]:
#Drops the appropriate rows
df_absorbance_scaled_encoded = df_scaled_encoded.drop(drop_list_abs)
#Saves the data for absorbance modelling to CSV
df_absorbance_scaled_encoded.to_csv('hao_all_abs_scaled.csv')